In [1]:
import re
from datetime import date
import pwlf
import datetime
import pickle
import seaborn as sns

import requests
import pandas as pd
from datetime import datetime
import time, math


# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('CllfFuOvEOBS09TAzabT9Q', 'UmOCgu0uvbPzHn6bJgPwZ5Yf6Rmx7w')

with open('pw.txt','r') as f:
    pw = f.read().rstrip('\n')
# here we pass our login method (password), username, and password
login = {'grant_type': 'password',
        'username': 'dspd1',
        'password': pw}



In [2]:
def get_headers(login):
    # setup our header info, which gives reddit a brief description of our app
    headers = {'User-Agent': 'MyAPI/0.0.1'}

    # send our request for an OAuth token
    res = requests.post('https://www.reddit.com/api/v1/access_token',
                        auth=auth, data=login, headers=headers)

    # convert response to JSON and pull access_token value
    TOKEN = res.json()['access_token']

    # add authorization to our headers dictionary
    headers['Authorization'] = f'bearer {TOKEN}'

    # while the token is valid (~2 hours) we just add headers=headers to our requests
    requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
    
    return headers

In [3]:
sub_reddits=['r/1P_LSD', 'r/2cb', 'r/4acodmt', 'r/adderall', 'r/afinil', 'r/ambien', 'r/AMT', 'r/Ayahuasca', 
'r/cocaine', 'r/cripplingalcoholism', 'r/DMT', 'r/DPH', 'r/dxm', 'r/fentanyl', 'r/ketamine', 'r/kratom', 
'r/LSA', 'r/LSD', 'r/MDMA', 'r/MemantineHCl', 'r/mescaline', 'r/meth', 'r/PCP', 
'r/phenibut', 'r/PsilocybinMushrooms', 'r/Salvia', 'r/shroomers', 'r/shrooms', 
'r/benzodiazepines', 'r/dissociatives', 'r/DissonautUniverse', 'r/gabagoodness', 
'r/microdosing', 'r/noids', 'r/opiates', 'r/Opioid_RCs', 'r/PsychedelicMessages', 
'r/Psychedelics', 'r/PsychedSubstance', 'r/Psychonaut', 'r/RationalPsychonaut', 'r/Stims', 
'r/treedibles', 'r/trees', 'r/tryptonaut', 'r/AnAnswerToHeal', 'r/AskDrugNerds', 'r/askdrugs', 'r/aves', 
'r/bestoferowid', 'r/CurrentlyTripping', 'r/druganalytics', 'r/druggardening', 'r/DrugNerds', 'r/Drugs', 
'r/DrugShowerThoughts', 'r/DrugsOver30', 'r/erowid', 'r/ObscureDrugs', 'r/ReagentTesting', 'r/researchchemicals', 
'r/samelevel', 'r/TheDrugClassroom', 'r/TheeHive', 'r/tripreports', 'r/TripSit', 'r/TripTales']

In [4]:
headers = get_headers(login)

In [5]:
def get_subreddit(subreddit, headers, params, dig_deeper):
    res = requests.get(f'https://oauth.reddit.com/{sub_reddit}/new',
                              headers=headers,
                              params=params)
    if 'error' in res.json():
        if res.json()['error']==403:
            print(f'private subreddit {sub_reddit}')
        elif res.json()['error']==404:
            reason = res.json()['reason']
            print(f'not found {sub_reddit}, reason {reason}')
        else:
            print(res.json())
        
        dig_deeper = False
    
    return res, dig_deeper

In [6]:
def get_comments_from_post(subreddit, post_id):
    df_from_comments = pd.DataFrame()

    res_comments = requests.get(f'https://oauth.reddit.com/{subreddit}/comments/{post_id}', 
               params={'depth':3,'sort':'old', 'limit':30}, headers=headers)
    try:
        if 'error' in res_comments.json():
            print(f'error in comments {res_comments.json()}')
    except:
        print(f' exception while checking error {res_comments}')
        
    try:
        if len(res_comments.json())>1:
            df_from_comments = df_comments(res_comments)
    except:
        print(res_comments)

    return df_from_comments

In [7]:
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = pd.concat([df, pd.Series({
            'subreddit': post['data']['subreddit'],
            'subreddit_name_prefixed': post['data']['subreddit_name_prefixed'],
            'name': post['data']['name'],
            'author': post['data']['author'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'num_comments':post['data']['num_comments'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'view_count': post['data']['view_count'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']}).to_frame().T], ignore_index=True)
        subreddit = post['data']['subreddit_name_prefixed']
        id_ = post['data']['id']
#         print(subreddit)
#         print(id_)
        df_from_comments = get_comments_from_post(subreddit=subreddit, post_id=id_)
        df = pd.concat([df, df_from_comments], ignore_index=True)
            
    return df

In [8]:
def df_comments(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()[1]['data']['children']:
        if post['kind']=='t1':
            df = pd.concat([df, pd.Series({
                'subreddit': post['data']['subreddit'],
                'subreddit_name_prefixed': post['data']['subreddit_name_prefixed'],
                'name': post['data']['name'],
                'author': post['data']['author'],
                'body': post['data']['body'],
                'ups': post['data']['ups'],
                'downs': post['data']['downs'],
                'score': post['data']['score'],
                'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
                'id': post['data']['id'],
                'kind': post['kind']}).to_frame().T], ignore_index=True)
    return df

# 'approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 
# 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 
# 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 
# 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 
# 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 
# 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 
# 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 
# 'author_flair_richtext', 'gildings', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type',
# 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html', 
# 'likes', 'suggested_sort', 'banned_at_utc', 'view_count', 'archived', 'no_follow', 'is_crosspostable', 'pinned', 
# 'over_18', 'all_awardings', 'awarders', 'media_only', 'can_gild', 'spoiler', 'locked', 'author_flair_text', 
# 'treatment_tags', 'visited', 'removed_by', 'num_reports', 'distinguished', 'subreddit_id', 'author_is_blocked', 
# 'mod_reason_by', 'removal_reason', 'link_flair_background_color', 'id', 'is_robot_indexable', 'report_reasons', 
# 'author', 'discussion_type', 'num_comments', 'send_replies', 'whitelist_status', 'contest_mode', 'mod_reports', 
# 'author_patreon_flair', 'author_flair_text_color', 'permalink', 'parent_whitelist_status', 'stickied', 'url', 
# 'subreddit_subscribers', 'created_utc', 'num_crossposts', 'media', 'is_video']

In [9]:
try:
    # prev_data = pd.read_csv('reddit.csv')
    # prev_data.set_index(['subreddit', 'id'], inplace=True)
    # display(prev_data)
    # Getting back the objects:
    with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        prev_data = pickle.load(f)
        
    load_from_begining = False
    print("Loaded previous data base - loading delta")

except:
    print(f'No previous database, loading from the beginning')
    prev_data = pd.DataFrame()
    load_from_begining = True


No previous database, loading from the beginning


In [10]:
try:
    prev_data_from_broken_loop=data.copy()
    prev_data_from_broken_loop.set_index(['subreddit', 'id'], inplace=True)
    print(f'prevous data frame already exists with {len(data)} entries, continue the broken loop')
    continue_broken_loop = True
    
except:
    data = pd.DataFrame()
    print('data initialised as empty')
    continue_broken_loop = False

start = time.time()
for sub_reddit in sub_reddits:
    
    params = {'limit': 100}
    print(sub_reddit)
    if load_from_begining==False:# set the last known data entry
        try:
            params['before'] = prev_data.loc[sub_reddit[2:]].iloc[0]['name']
            i=0
        except:
            print(f'No {sub_reddit} in previous data set, continue to the next one')
            continue
    elif continue_broken_loop: # loop was broken
        try:
            params['after'] = prev_data_from_broken_loop.loc[prev_data_from_broken_loop['body'].isna()].loc[sub_reddit[2:]].iloc[-1]['name']
            i = len(prev_data_from_broken_loop.loc[prev_data_from_broken_loop['body'].isna()].loc[sub_reddit[2:]])/100
            i = math.floor(i)
            print(i)
        except:
            print(f'No {sub_reddit} in prev data set, loading from the begining')
            params = {'limit': 100}
            i=0
    else:
        i=0

    year_of_comments = datetime.now().year
    dig_deeper = True
    while dig_deeper:
        i += 1
        dig_deeper = i<=100 | year_of_comments>2010

        res, dig_deeper = get_subreddit(sub_reddit, headers, params, dig_deeper)
        if dig_deeper==False:
            print(f'get subreddit returned dig_deeper false, stopped at i={i}')
            continue
        

        new_df = df_from_response(res) #create new data frame
        if len(new_df)==0:
            dig_deeper=False
            print(f'empty res, stopped at i={i}')
            continue
            
        
        if load_from_begining:
            # take the final row (oldest entry) not the comments
            if 'body' in new_df:
                row = new_df.loc[new_df['body'].isna()].iloc[-1] 
            else:
                row = new_df.iloc[-1]             
            params['after'] = row['name']
            a = pd.to_datetime(row['created_utc'])
            year_of_comments = a.year

        else:
            # take the first row (newest entry)
            if 'body' in new_df:
                row = new_df.loc[new_df['body'].isna()].iloc[0]              
            else:
                row = new_df.iloc[0]
            # add/update fullname in params
            params['before'] = row['name']
          

        # append new_df to data
        data = pd.concat([data, new_df], ignore_index=True)
end = time.time()
print('time to scrap',end - start)


data initialised as empty
r/1P_LSD
empty res, stopped at i=11
r/2cb
empty res, stopped at i=11
r/4acodmt
empty res, stopped at i=9
r/adderall
empty res, stopped at i=3
r/afinil
empty res, stopped at i=11
r/ambien
empty res, stopped at i=11
r/AMT
empty res, stopped at i=2
r/Ayahuasca
empty res, stopped at i=11
r/cocaine
empty res, stopped at i=11
r/cripplingalcoholism
private subreddit r/cripplingalcoholism
get subreddit returned dig_deeper false, stopped at i=1
r/DMT
 exception while checking error <Response [504]>
<Response [504]>
empty res, stopped at i=11
r/DPH
empty res, stopped at i=11
r/dxm
empty res, stopped at i=11
r/fentanyl
empty res, stopped at i=11
r/ketamine
empty res, stopped at i=11
r/kratom
empty res, stopped at i=11
r/LSA
empty res, stopped at i=11
r/LSD
empty res, stopped at i=11
r/MDMA
empty res, stopped at i=11
r/MemantineHCl
empty res, stopped at i=10
r/mescaline
empty res, stopped at i=11
r/meth
empty res, stopped at i=11
r/PCP
empty res, stopped at i=5
r/phenibut

In [11]:
if load_from_begining:
    data_combined_with_delta = data
else:
    data_combined_with_delta = pd.concat([data, prev_data.reset_index()], ignore_index=True)
    
data_combined_with_delta.sort_values(by=['subreddit', 'created_utc'], ascending=(True,False), inplace=True)
data_combined_with_delta.set_index(['subreddit', 'id'], inplace=True)

display(data_combined_with_delta)

subreddit_name_prefixed        name                author  \
subreddit   id                                                                  
1P_LSD      j56fqvn                r/1P_LSD  t1_j56fqvn             Miley-Jay   
            j55zgsj                r/1P_LSD  t1_j55zgsj              nowayfly   
            j55xsbx                r/1P_LSD  t1_j55xsbx  Environmental-Fan196   
            j55vtlh                r/1P_LSD  t1_j55vtlh    Potato-Interesting   
            j55q3nc                r/1P_LSD  t1_j55q3nc   Realistic_Froyo_952   
...                                     ...         ...                   ...   
tripreports otqz2r            r/tripreports   t3_otqz2r              anusmold   
            h6x7m57           r/tripreports  t1_h6x7m57         AutoModerator   
            h6px1cc           r/tripreports  t1_h6px1cc             Psych_Art   
            osohl1            r/tripreports   t3_osohl1      OtherwiseWePanic   
            h6ppewh           r/tripreports  t1_h6ppewh         AutoModerator   

                                                        title  \
subreddit   id                                                  
1P_LSD      j56fqvn                                       NaN   
            j55zgsj                                       NaN   
            j55xsbx                                       NaN   
            j55vtlh                                       NaN   
            j55q3nc                                       NaN   
...                                                       ...   
tripreports otqz2r   Forgot I took DMT during the trip itself   
            h6x7m57                                       NaN   
            h6px1cc                                       NaN   
            osohl1                          1P-LSD + 4-HO-MET   
            h6ppewh                                       NaN   

                                                              selftext  \
subreddit   id                                                           
1P_LSD      j56fqvn                                                NaN   
            j55zgsj                                                NaN   
            j55xsbx                                                NaN   
            j55vtlh                                                NaN   
            j55q3nc                                                NaN   
...                                                                ...   
tripreports otqz2r   So I believe I had my first breakthrough. I di...   
            h6x7m57                                                NaN   
            h6px1cc                                                NaN   
            osohl1   1P-LSD + 4-HO-MET\n\n**(300ug 1P-LSD with 50mg...   
            h6ppewh                                                NaN   

                    num_comments upvote_ratio ups downs view_count score  \
subreddit   id                                                             
1P_LSD      j56fqvn          NaN          NaN   1     0        NaN     1   
            j55zgsj          NaN          NaN  -3     0        NaN    -3   
            j55xsbx          NaN          NaN   0     0        NaN     0   
            j55vtlh          NaN          NaN   3     0        NaN     3   
            j55q3nc          NaN          NaN   2     0        NaN     2   
...                          ...          ...  ..   ...        ...   ...   
tripreports otqz2r             3          1.0   1     0       None     1   
            h6x7m57          NaN          NaN   1     0        NaN     1   
            h6px1cc          NaN          NaN   4     0        NaN     4   
            osohl1             9          1.0  14     0       None    14   
            h6ppewh          NaN          NaN   1     0        NaN     1   

                    link_flair_css_class           created_utc kind  \
subreddit   id                                                        
1P_LSD      j56fqvn       

In [12]:
#data_combined_with_delta.to_csv('reddit.csv')  
import os

try:
    os.system('cp objs.pkl objs.pkl.bkp')
except:
    print('no previous version found')


cp: cannot stat 'objs.pkl': No such file or directory


256

In [15]:
# Saving the objects:
data_combined_with_delta['year_month'] = \
     pd.to_datetime(pd.to_datetime(data_combined_with_delta['created_utc']).dt.strftime('%Y-%m'))
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(data_combined_with_delta, f)

